# Выбор локации для скважины

Для нефтедобывающей компании нужно ответить на вопрос: "где бурить новую скважину?".

В таблице с данными хранятся записи о пробах нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов.

Задачи: 
1. Построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль.
2. Проанализировать возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

1. В избранном регионе найти месторождения. Для каждого определить значения признаков.
2. Построить модель и оценить объём запасов.
3. Выбрать месторождения с самыми высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины.
4. Посчитать прибыль. Она равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
from scipy import stats as st
from pandas.plotting import scatter_matrix 
import matplotlib.pyplot as plt

In [2]:
df0 = pd.read_csv('/datasets/geo_data_0.csv')
df1 = pd.read_csv('/datasets/geo_data_1.csv')
df2 = pd.read_csv('/datasets/geo_data_2.csv')


### df0

In [3]:
display(df0)
df0.info()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...,...
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Удалим столбец id, для того чтобы не было ошибок при обучении модели Логистической Регрессии.
Проверим тип данных в столбцах.
Проверим на наличие пропусков и дубликатов.

In [4]:
print('Дубликаты:', df0.duplicated().sum())

Дубликаты: 0


In [5]:
print('Пропуски:')
print(df0.isna().sum())

Пропуски:
id         0
f0         0
f1         0
f2         0
product    0
dtype: int64


In [6]:
df0= df0.drop('id', axis=1)

### df1

In [7]:
display(df1)
df1.info()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
...,...,...,...,...,...
99995,QywKC,9.535637,-6.878139,1.998296,53.906522
99996,ptvty,-10.160631,-12.558096,5.005581,137.945408
99997,09gWa,-7.378891,-3.084104,4.998651,137.945408
99998,rqwUm,0.665714,-6.152593,1.000146,30.132364


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [8]:
print('Дубликаты:', df1.duplicated().sum())

Дубликаты: 0


In [9]:
print('Пропуски:')
print(df1.isna().sum())

Пропуски:
id         0
f0         0
f1         0
f2         0
product    0
dtype: int64


In [10]:
df1= df1.drop('id', axis=1)

### df2

In [11]:
display(df2)
df2.info()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
...,...,...,...,...,...
99995,4GxBu,-1.777037,1.125220,6.263374,172.327046
99996,YKFjq,-1.261523,-0.894828,2.524545,138.748846
99997,tKPY3,-1.199934,-2.957637,5.219411,157.080080
99998,nmxp2,-2.419896,2.417221,-5.548444,51.795253


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [12]:
print('Дубликаты:', df2.duplicated().sum())

Дубликаты: 0


In [13]:
print('Пропуски:')
print(df2.isna().sum())

Пропуски:
id         0
f0         0
f1         0
f2         0
product    0
dtype: int64


In [14]:
df2= df2.drop('id', axis=1)

### Соединение таблиц в список

In [15]:
all_data = [df0, df1, df2]

## Обучение и проверка модели

### Функция для расчета среднего запаса сырья в регионе:

In [16]:
def average_oil_reserve_and_rmse(data):
    target = data['product']
    features = data.drop('product', axis=1)
    features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)
    
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)

    predictions_valid = pd.Series(predictions_valid)
    target_valid = target_valid.reset_index(drop=True)

    mean_product = predictions_valid.sum() / len(predictions_valid)

    rmse = mean_squared_error(target_valid, predictions_valid)**0.5
    
    
    return f'Средний запас сырья в регионе:, {mean_product}, RMSE модели региона:, {rmse}'

### Расчет среднего запаса сырья в регионе:

In [17]:
count = 0
for i in all_data:
    print('Регион', count, ':')
    print(average_oil_reserve_and_rmse(i))
    count += 1

Регион 0 :
Средний запас сырья в регионе:, 92.59256778438035, RMSE модели региона:, 37.5794217150813
Регион 1 :
Средний запас сырья в регионе:, 68.728546895446, RMSE модели региона:, 0.893099286775617
Регион 2 :
Средний запас сырья в регионе:, 94.96504596800489, RMSE модели региона:, 40.02970873393434


Самый большой средни запас сырья наблюдается в регионе 2.

## Подготовка к расчёту прибыли

**Цена разработки 200 скважин в регионе:**

In [18]:
price = 10000000000

**Стоимость одной тысячи баррелей нефти:**

In [19]:
barrel_price = 450000

**Количество разрабатываемых в регионе скважин:**

In [20]:
amount = 200

**Средний запас сырья в регионе для безубыточной разработки одной скважины:**

In [21]:
print((price / amount) / barrel_price, 'тыс. баррелей нефти')

111.11111111111111 тыс. баррелей нефти


### Функция расчета прибыли:

In [22]:
def profit(target, predictions):
    pred_sorted = predictions.sort_values(ascending=False)
    selected = target[pred_sorted.index][:amount]
    return barrel_price * selected.sum() - price

## Расчёт прибыли и рисков 

### Функция расчета прибыли и рисков 

In [23]:
def profit_and_risks(data):
    target = data['product']
    features = data.drop('product', axis=1)
    features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)
    
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)

    predictions_valid = pd.Series(predictions_valid)
    target_valid = target_valid.reset_index(drop=True)
    
    state = np.random.RandomState(123456)

    values = []
    for i in range(1000):
        target_subsample = target_valid.sample(n=500, replace=True, random_state=state)
        pred_subsample = predictions_valid[target_subsample.index] 

        values.append(profit(target_subsample, pred_subsample))

    values = pd.Series(values)
        
    confidence_interval = (values.quantile(0.025), values.quantile(0.975))
    
    count = 0
    for number in values:
        if number < 0:
            count += 1
    risks = count * 100 / len(values)
    
    return f'Средняя прибыль: {values.mean()}, 95%-ый доверительный интервал для ПРОГНОЗА: {confidence_interval},   риск: {risks}%'


### Расчет средней прибыли для региона и доверительных интервалов

In [24]:
count = 0
for i in all_data:
    print('Регион', count, ':')
    print(profit_and_risks(i))
    count += 1

Регион 0 :
Средняя прибыль: 442013608.40307546, 95%-ый доверительный интервал для ПРОГНОЗА: (-99960168.53370465, 969184228.5952274),   риск: 5.6%
Регион 1 :
Средняя прибыль: 515921153.1698947, 95%-ый доверительный интервал для ПРОГНОЗА: (105927220.97361174, 956491279.345911),   риск: 0.7%
Регион 2 :
Средняя прибыль: 415763796.1457504, 95%-ый доверительный интервал для ПРОГНОЗА: (-129560776.43486977, 972242514.01586),   риск: 7.4%


Лучший регион для разработки - 1. Вероятность убытков в нем меньше 2.5%.

## Вывод:

Лучшим регионом признан регион 1, вероятность убытков примерно 0.7%, средняя прибыль составляет 515921153.1698947.